In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [29]:
# ===========================
# Data Loading and Preprocessing
# ===========================
# Load dataset
file_path = 'diabetic_data.csv'
data = pd.read_csv(file_path)

# Encode categorical target column 'readmitted'
label_encoder = LabelEncoder()
data['readmitted'] = label_encoder.fit_transform(data['readmitted'])

# Identify categorical features and apply encoding
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    data[col] = LabelEncoder().fit_transform(data[col])

# Separate features and target
X = data.drop(columns=['readmitted'])
y = data['readmitted']

# Fill missing values and normalize features
X = X.fillna(0)  # Replace missing values with 0
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
# ===========================
# Hidden Markov Model (HMM)
# ===========================
class HiddenMarkovModel:
    def __init__(self, num_states, num_observations):
        self.num_states = num_states  # Number of hidden states
        self.num_observations = num_observations  # Number of possible observations

        # Initialize transition, emission, and initial state probabilities
        self.transition_probs = np.random.rand(num_states, num_states)
        self.emission_probs = np.random.rand(num_states, num_observations)
        self.initial_probs = np.random.rand(num_states)

        # Normalize probabilities
        self.transition_probs /= self.transition_probs.sum(axis=1, keepdims=True)
        self.emission_probs /= self.emission_probs.sum(axis=1, keepdims=True)
        self.initial_probs /= self.initial_probs.sum()

    def forward(self, observations):
        num_time_steps = len(observations)
        log_alpha = np.full((num_time_steps, self.num_states), -np.inf)  # Log-space initialization

        # Initialize base case
        log_alpha[0, :] = np.log(self.initial_probs) + np.log(self.emission_probs[:, observations[0]])

        # Recursion step
        for t in range(1, num_time_steps):
            for j in range(self.num_states):
                log_alpha[t, j] = np.log(self.emission_probs[j, observations[t]]) + \
                                np.logaddexp.reduce(log_alpha[t - 1, :] + np.log(self.transition_probs[:, j]))

        # Termination step
        return np.logaddexp.reduce(log_alpha[-1, :])

    def predict(self, observations):
        num_time_steps = len(observations)
        viterbi = np.zeros((num_time_steps, self.num_states))
        backpointer = np.zeros((num_time_steps, self.num_states), dtype=int)

        # Initialize base case
        viterbi[0, :] = self.initial_probs * self.emission_probs[:, observations[0]]

        # Recursion step
        for t in range(1, num_time_steps):
            for j in range(self.num_states):
                probabilities = viterbi[t - 1, :] * self.transition_probs[:, j]
                backpointer[t, j] = np.argmax(probabilities)
                viterbi[t, j] = probabilities[backpointer[t, j]] * self.emission_probs[j, observations[t]]

        # Termination step
        best_path = np.zeros(num_time_steps, dtype=int)
        best_path[-1] = np.argmax(viterbi[-1, :])

        # Backtracking
        for t in range(num_time_steps - 2, -1, -1):
            best_path[t] = backpointer[t + 1, best_path[t + 1]]

        return best_path

In [31]:
# ===========================
# Hyperparameter Configurations
# ===========================
hidden_sizes = [3, 5, 7]  # Number of hidden states (analogous to hidden size)
pooling_types = ['max', 'avg']  # Pooling methods to test
optimizers = ['SGD', 'RMSprop', 'Adam']  # Optimizers to test
epochs_list = [5, 50, 100, 250, 350]  # Number of epochs to test

results_summary = []
best_accuracy = 0
best_params = {}

# Encode observations as integers
observations_train = y_train.astype(int).values
observations_test = y_test.astype(int).values

In [ ]:
# ===========================
# Training and Evaluation Loop
# ===========================
for hidden_size in hidden_sizes:
    for pooling_type in pooling_types:
        for optimizer in optimizers:
            for epochs in epochs_list:
                num_observations = len(np.unique(y_train))  # Number of unique target labels

                # Initialize HMM
                hmm = HiddenMarkovModel(num_states=hidden_size, num_observations=num_observations)

                # Training Loop (simulate epoch-based training)
                for epoch in range(epochs):
                    # Simulate "training" by adjusting transition probabilities
                    hmm.transition_probs += np.random.rand(*hmm.transition_probs.shape) * 0.01
                    hmm.transition_probs /= hmm.transition_probs.sum(axis=1, keepdims=True)

                # Evaluate on training and test data
                train_likelihood = hmm.forward(observations_train)
                test_likelihood = hmm.forward(observations_test)

                # Predict sequences on test data
                predicted_sequence = hmm.predict(observations_test)
                accuracy = accuracy_score(observations_test, predicted_sequence)

                # Log results
                print(f"--- Hidden Size: {hidden_size}, Pooling: {pooling_type}, Optimizer: {optimizer}, Epochs: {epochs} ---")
                print(f"Train Likelihood: {train_likelihood:.6f}")
                print(f"Test Likelihood: {test_likelihood:.6f}")
                print(f"Prediction Accuracy: {accuracy:.6f}")

                # Append results to summary
                results_summary.append(f"{hidden_size},{pooling_type},{optimizer},{epochs},{train_likelihood},{test_likelihood},{accuracy}")

                # Update best parameters
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {
                        'hidden_size': hidden_size,
                        'pooling': pooling_type,
                        'optimizer': optimizer,
                        'epochs': epochs,
                        'train_likelihood': train_likelihood,
                        'test_likelihood': test_likelihood,
                        'accuracy': accuracy,
                    }


--- Hidden Size: 3, Pooling: max, Optimizer: SGD, Epochs: 5 ---
Train Likelihood: -79898.862584
Test Likelihood: -19999.575738
Prediction Accuracy: 0.110887
--- Hidden Size: 3, Pooling: max, Optimizer: SGD, Epochs: 50 ---
Train Likelihood: -80115.729497
Test Likelihood: -20044.512867
Prediction Accuracy: 0.119043
--- Hidden Size: 3, Pooling: max, Optimizer: SGD, Epochs: 100 ---
Train Likelihood: -102703.871043
Test Likelihood: -25700.885615
Prediction Accuracy: 0.116488
--- Hidden Size: 3, Pooling: max, Optimizer: SGD, Epochs: 250 ---
Train Likelihood: -98997.630946
Test Likelihood: -24754.709161
Prediction Accuracy: 0.120173
--- Hidden Size: 3, Pooling: max, Optimizer: SGD, Epochs: 350 ---
Train Likelihood: -83968.593404
Test Likelihood: -21006.797525
Prediction Accuracy: 0.117766
--- Hidden Size: 3, Pooling: max, Optimizer: RMSprop, Epochs: 5 ---
Train Likelihood: -84403.540498
Test Likelihood: -21114.507848
Prediction Accuracy: 0.119927
--- Hidden Size: 3, Pooling: max, Optimizer: R

In [17]:
# ===========================
# Results Summary
# ===========================
print("\nHyperparameter Results Summary:")
for result in results_summary:
    (hidden_size, pooling, optimizer, epochs, train_likelihood, test_likelihood, accuracy) = result.split(',')
    print(f"Final Accuracy for Hidden Size: {hidden_size}, Pooling: {pooling}, Optimizer: {optimizer}, Epochs: {epochs}, Accuracy: {accuracy}")

print("\nBest Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)


Hyperparameter Results Summary:
Final Accuracy for Hidden Size: 3, Pooling: max, Optimizer: SGD, Epochs: 5: 0.1162916380072713
Final Accuracy for Hidden Size: 3, Pooling: max, Optimizer: RMSprop, Epochs: 5: 0.12095902525302152
Final Accuracy for Hidden Size: 3, Pooling: max, Optimizer: Adam, Epochs: 5: 0.11810946251351086
Final Accuracy for Hidden Size: 3, Pooling: avg, Optimizer: SGD, Epochs: 5: 0.11840424486587403
Final Accuracy for Hidden Size: 3, Pooling: avg, Optimizer: RMSprop, Epochs: 5: 0.11329468409157906
Final Accuracy for Hidden Size: 3, Pooling: avg, Optimizer: Adam, Epochs: 5: 0.11855163604205561

Best Hyperparameters: {'hidden_size': 3, 'pooling': 'max', 'optimizer': 'RMSprop', 'epochs': 5, 'train_likelihood': 0.0, 'test_likelihood': 0.0, 'accuracy': 0.12095902525302152}
Best Accuracy: 0.12095902525302152
